In [1]:
%pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [4]:
! npm install localtunnel

"npm" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [5]:
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [9]:
%%writefile app.py 
#Creamos el archivo de la APP en el interprete principal (Phyton)
#####################################################
#Importamos librerias
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
######################################################  
#Definimos la instancia
@st.cache_resource
######################################################
#Creamos la función de carga de datos
def load_data():
   #Lectura del archivo csv
   df=pd.read_csv("Datos_limpios.csv")
    #Rellenamos nulos
   df =df.fillna(method="bfill")
   df =df.fillna(method="ffill")  
   Lista=['property_type', 'price', 'review_scores_rating', 'estimated_revenue_l365d','host_is_superhost','estimated_occupancy_l365d','beds','calculated_host_listings_count','accommodates','review_scores_value','room_type']
   return df, Lista
###############################################################################
#Cargo los datos obtenidos de la función "load_data"
df, Lista = load_data()
###############################################################################
#CREACIÓN DEL DASHBOARD
#Generamos las páginas que utilizaremos en el diseño
##############################################################################
#Generamos los encabezados para la barra lateral (sidebar)
st.sidebar.title("Airbnb México")

st.markdown(
   """
    <style>
    .stApp {
        background-color: white; /* Cambia el color aquí */
    }
    h1 {
        font-size: 40px;
        color: blue; /* Cambia el color aquí */
        font-weight: bold;
        text-align: left; /* Alinear a la izquierda el texto */
        }
    h2 {
        font-size: 24px;
        color: blue; /* Cambia el color aquí */
        font-weight: bold;
        text-align: left; /* Alinear a la izquierda el texto */
        }
    h3 {
        font-size: 12px;
        color: blue; /* Cambia el color aquí */
        font-weight: bold;
        text-align: left; /* Alinear a la izquierda el texto */
        }
    </style>
    """,
    unsafe_allow_html=True
)
############################################################################################
#Customización de la sidebar
st.sidebar.markdown(
    """
    <style>
    .stSidebar {
        background-color: #FF385C; /* Cambia el color aquí */
    }    
    .stSidebar h1 {
        font-size: 30px;
        color: white; /* Cambia el color aquí */
        font-weight: bold;
        text-align: center; /* Centrar el texto */
    }
    .stSidebar h2 {
        font-size: 24px;
        color: blue; /* Cambia el color aquí */
        font-weight: bold;
        text-align: left; /* Alinear a la izquierda el texto */
    }
    .stSidebar h3 {
        font-size: 18px;
        color: black; /* Cambia el color aquí */
        font-weight: bold;
        text-align: left; /* Alinear a la izquierda el texto */
    }
    .stButton > button {
        background-color: red; /* Cambia el color de fondo */
        color: white; /* Cambia el color del texto */
        padding: 10px 20px; /* Añade algo de padding */
        border: none; /* Sin bordes */
        border-radius: 5px; /* Bordes redondeados */
        cursor: pointer; /* Cambia el cursor */
    }
    .stButton > button:hover {
        background-color: green; /* Cambia el color al pasar el ratón */
        color: white; /* Cambia el color del texto */
    }
    </style>
    """,
    unsafe_allow_html=True
)
st.sidebar.image("logoairbnbblancosinfondo.png", width=90)

st.image("ciudaddemexico.jpg", width= 200)

#Widget 1: Selectbox
#Menu desplegable de opciones de las páginas seleccionadas
View= st.sidebar.selectbox(label= "Tipo de Análisis", options= ["Extracción de Características", 
                                              "Regresión Lineal", "Regresión No Lineal",
                                              "Regresión Logística"])
                                            



# CONTENIDO DE LA VISTA 1
if View == "Extracción de Características":
    #EXTRACCIÓN DE CARACTERÍSTICAS
    #Select box
    Variable_Cat= st.sidebar.selectbox(label= "Variables", options= Lista)
    #Obtenemos las frecuencias de las categorías de la variable seleccionada
    Tabla_frecuencias = df[Variable_Cat].value_counts().reset_index()
    #Ajustamos los nombre de las cabeceras de las columnas
    Tabla_frecuencias.columns = ['categorias', 'frecuencia']
   
    options = ["Barras", "Pastel", "Anillo", "Área"]
    selection = st.sidebar.pills("Tipo Gráfica", options, selection_mode="multi")
    st.markdown(f"Selección: {selection}.")
    
    #Generamos los encabezados para el dashboard
    st.title("Extracción de Características")

    if "Barras" in selection:
        st.write("Grafico de Barras")
        #GRAPH 1: BARPLOT
        #Despliegue de un bar plot, definiendo las variables "X categorias" y "Y numéricas" 
        Tabla_frecuencias_top10 = Tabla_frecuencias.sort_values(by='frecuencia', ascending=False).head(10)

        figure1 = px.bar(data_frame=Tabla_frecuencias_top10, x='categorias',

                  y= 'frecuencia', title= str('Frecuencia por categoría'))
        
        figure1.update_xaxes(automargin=True)

        figure1.update_yaxes(automargin=True)

        figure1.update_layout(height=300)

        st.plotly_chart(figure1, use_container_width=True)
    

    if "Pastel" in selection:
        st.write("Grafico de Pastel")
        #GRAPH 2: PIEPLOT
        #Despliegue de un pie plot, definiendo las variables "X categorias" y "Y numéricas" 
        Tabla_frecuencias = Tabla_frecuencias.sort_values(by='frecuencia', ascending=False).head(5)
        figure2 = px.pie(data_frame=Tabla_frecuencias, names='categorias', values='frecuencia', title='Frecuencia por categoría')
        figure2.update_layout(height=300)
        st.plotly_chart(figure2, use_container_width=True)


    if "Anillo" in selection:
        st.write("Grafico de anillo")
        #GRAPH 3: DONUT PLOT
        #Despliegue de un line plot, definiendo las variables "X categorias" y "Y numéricas" 
        Tabla_frecuencias = Tabla_frecuencias.sort_values(by='frecuencia', ascending=False).head(5)
        figure3 = px.pie(data_frame=Tabla_frecuencias, names='categorias', values='frecuencia', hole=0.4, title='Frecuencia por categoría (Top 5)')
        figure3.update_layout(height=300)
        st.plotly_chart(figure3, use_container_width=True)
        

    if "Área" in selection:
        st.write("Grafico de area")
        #GRAPH 4: AREA PLOT
        #Despliegue de un area plot, definiendo las variables "X categorias" y "Y numéricas" 
        figure4 = px.area(data_frame=Tabla_frecuencias, x='categorias', 
                  y= 'frecuencia', title= str('Frecuencia por categoría'))
        figure4.update_layout(height=300)
        st.plotly_chart(figure4, use_container_width=True)
        

 ###################################################################################
 
 # CONTENIDO DE LA VISTA 2
if View == "Regresión Lineal":
    #REGRESIÓN LINEAL SIMPLE 
    #Generamos la lista de variables numéricas
    numeric_df = df.select_dtypes(['float','int'])  #Devuelve Columnas
    Lista_num= numeric_df.columns                   #Devuelve lista de Columnas numéricas
    #Select box
    Variable_y= st.sidebar.selectbox(label= "Variable objetivo (Y)", options= Lista_num)
    Variable_x= st.sidebar.selectbox(label= "Variable independiente del modelo simple (X)", options= Lista_num)    
    
    
    #Generamos los encabezados para el dashboard
    st.title("Regresión Lineal")  
    
    #Generamos el diseño del Layout deseado
    # Fila 1
    Contenedor_A, Contenedor_B = st.columns(2)
    with Contenedor_A: 
        st.write(f"Correlación lineal simple: {Variable_y}")
      
        #Se define model como la función de regresión lineal
        from sklearn.linear_model import LinearRegression
        model= LinearRegression()
        #Ajustamos el modelo con las variables antes declaradas
        model.fit(X=df[[Variable_x]], y=df[Variable_y])
        #Predecimos los valores de la variable objetivo
        y_pred= model.predict(X=df[[Variable_x]])
        #Corroboramos cual es el coeficiente de Determinación de nuestro modelo
        coef_Deter_simple=model.score(X=df[[Variable_x]], y=df[Variable_y])
        #Corroboramos cual es el coeficiente de Correlación de nuestro modelo
        coef_Correl_simple=np.sqrt(coef_Deter_simple)
        #Mostramos el dataset
        st.write(coef_Correl_simple)
        
        #GRAPH 5: SCATTERPLOT
        import plotly.express as px
        import plotly.graph_objects as go
        
        figure5 = px.scatter(data_frame=numeric_df, x=Variable_x, y=Variable_y, 
                             title= 'Modelo Lineal Simple')
        
        figure5.add_trace(
            go.Scatter(
                x=df[Variable_x], 
                y=y_pred,
                mode='markers',
                name='Línea de Regresión',
                line=dict(color='red', width=2)
            )
        )
        st.plotly_chart(figure5)
    
    with Contenedor_B:
        st.write(f"Correlación Lineal Múltiple:{Variable_y}")
        #Widget 3: Multiselect box
        #Generamos un cuadro de multi-selección (X) para seleccionar variables independientes
        Variables_x= st.sidebar.multiselect(label="Variables independientes del modelo múltiple (X)", options= Lista_num)
        #Se define model como la función de regresión lineal
        from sklearn.linear_model import LinearRegression
        model_M= LinearRegression()
        #Ajustamos el modelo con las variables antes declaradas
        model_M.fit(X=df[Variables_x], y=df[Variable_y])
        #Predecimos los valores de la variable objetivo
        y_pred_M= model_M.predict(X=df[Variables_x])
        #Corroboramos cual es el coeficiente de Determinación de nuestro modelo
        coef_Deter_multiple=model_M.score(X=df[Variables_x], y=df[Variable_y])
        #Corroboramos cual es el coeficiente de Correlación de nuestro modelo
        coef_Correl_multiple=np.sqrt(coef_Deter_multiple)
        #Mostramos el coeficiente de correlación múltiple
        st.write(coef_Correl_multiple)
        
        #GRAPH 6: SCATTERPLOT
        figure6 = px.scatter(data_frame=numeric_df, x=Variables_x, y=Variable_y,
                     title= 'Modelo Lineal Múltiple')

        figure6.add_trace(
            go.Scatter(
                x=df[Variables_x], 
                y=y_pred_M,
                mode='markers',
                name='Línea de Regresión',
                line=dict(color='red', width=2)
            )
        )
        st.plotly_chart(figure6)
        
 ####################################################################################ç
 
  # CONTENIDO DE LA VISTA 3
if View == "Regresión No Lineal":
    #REGRESIÓN NO LINEAL 
    #Generamos la lista de variables numéricas
    numeric_df = df.select_dtypes(['float','int'])  #Devuelve Columnas
    df_nosuperhost= df[df["host_is_superhost"] == 0]
    df_superhost=df[df["host_is_superhost"] == 1]
    Lista_num= numeric_df.columns                   #Devuelve lista de Columnas numéricas
    #Select box
    Variable_y= st.sidebar.selectbox(label= "Variable objetivo (Y)", options= Lista_num)
    Variable_x= st.sidebar.selectbox(label= "Variable independiente del modelo No lineal (X)", options= Lista_num)   
    #Lista de modelos
    Lista_mod=["Función cuadrática", "Función exponencial","Función inversa","Función senoidal","Función tangencial","Función Valor absoluto","Función cociente entre polinomios","Función logaritmica","Función lineal con producto de coeficientes","Función cuadrática inversa","Función polinomial inversa"]
    #Select box de modelos
    Modelo= st.sidebar.selectbox(label= "Modelos No Lineales", options= Lista_mod)  

    #Generamos los encabezados para el dashboard
    st.title("Regresión No Lineal")  
        
        
    #Generamos el diseño del Layout deseado
    # Fila 1
    Contenedor_A, Contenedor_B = st.columns(2)
    with Contenedor_A: 
        st.write(f"Correlación No lineal No Superhost: {Variable_y}")
        #Redefinimos las variables 
        x= df_nosuperhost[Variable_x]
        y= df_nosuperhost[Variable_y]
        
        #Declaración de MODELOS
        if Modelo == "Función cuadrática":
            def  func1 (x, a, b, c):
                return a*x**2 + b*x + c
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func1, x, y)
            # Predicciones
            y_pred = func1(x, *parametros)
        
        if Modelo == "Función exponencial":
            def func2(x, a, b, c):
                return a*np.exp(-b*x)+c
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func2, x, y)
            # Predicciones
            y_pred = func2(x, *parametros)

        if Modelo == "Función inversa":
            def func3(x, a, b, c):
                return 1/a*x
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func3, x, y)
            # Predicciones
            y_pred = func3(x, *parametros)

        if Modelo == "Función senoidal":
            def func4(x, a, b, c):
                return a*np.sin(x) + b
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func4, x, y)
            # Predicciones
            y_pred = func4(x, *parametros)

        if Modelo == "Función tangencial":
            def func5(x, a, b, c):
                return a*np.tan(x) + b
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func5, x, y)
            # Predicciones
            y_pred = func5(x, *parametros)

        if Modelo == "Función valor absoluto":
            def func6(x, a, b, c):
                return a*np.abs(x) + b*x + c
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func6, x, y)
            # Predicciones
            y_pred = func6(x, *parametros)

        if Modelo == "Función cociente entre polinomios":
            def func7(x, a, b, c):
                return a*np.abs(x) + b*x + c
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func7, x, y)
            # Predicciones
            y_pred = func7(x, *parametros)

        if Modelo == "Función logaritmica":
            def func8(x, a, b, c):
                return a*np.log(x) + b
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func8, x, y)
            # Predicciones
            y_pred = func8(x, *parametros)

        if Modelo == "Función lineal con producto de coeficientes":
            def func9(x, a, b, c):
                return a*x + b*x + c*x
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func9, x, y)
            # Predicciones
            y_pred = func9(x, *parametros)

        if Modelo == "Función cuadrática inversa":
            def func10(x, a, b, c):
                return 1/a*x**2
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func10, x, y)
            # Predicciones
            y_pred = func10(x, *parametros)

        if Modelo == "Función polinomial inversa":
            def func11(x, a, b, c):
                return a/b*x**2 + c*x
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func11, x, y)
            # Predicciones
            y_pred = func11(x, *parametros)

        #Corroboramos cual es el coeficiente de Determinación de nuestro modelo
        coef_Deter = r2_score(y, y_pred) 
        #Corroboramos cual es el coeficiente de Correlación de nuestro modelo
        coef_Correl=np.sqrt(coef_Deter)
        #Mostramos el dataset
        st.write(coef_Correl)
        
        #GRAPH 7: SCATTERPLOT
        figure7 = px.scatter(data_frame=df_nosuperhost, x=Variable_x, y=Variable_y, 
                     title= 'Modelo No Lineal no Superhost')

        figure7.add_trace(
            go.Scatter(
                x=df[Variable_x], 
                y=y_pred,
                mode='markers',
                name='Línea de Regresión',
                line=dict(color='red', width=2)
            )
        )
        st.plotly_chart(figure7)

    with Contenedor_B: 
        st.write(f"Correlación No lineal Superhost:  {Variable_y}")
        #Redefinimos las variables 
        x= df_superhost[Variable_x]
        y= df_superhost[Variable_y]
               #Declaración de MODELOS
        if Modelo == "Función cuadrática":
            def  func1 (x, a, b, c):
                 return a*x**2 + b*x + c
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func1, x, y)
            # Predicciones
            y_pred = func1(x, *parametros)
    
        if Modelo == "Función exponencial":
            def func2(x, a, b, c):
                return a*np.exp(-b*x)+c
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func2, x, y)
            # Predicciones
            y_pred = func2(x, *parametros)

        if Modelo == "Función inversa":
            def func3(x, a, b, c):
                return 1/a*x
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func3, x, y)
            # Predicciones
            y_pred = func3(x, *parametros)

        if Modelo == "Función senoidal":
            def func4(x, a, b, c):
                return a*np.sin(x) + b
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func4, x, y)
            # Predicciones
            y_pred = func4(x, *parametros)

        if Modelo == "Función tangencial":
            def func5(x, a, b, c):
                return a*np.tan(x) + b
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func5, x, y)
            # Predicciones
            y_pred = func5(x, *parametros)

        if Modelo == "Función valor absoluto":
            def func6(x, a, b, c):
                return a*np.abs(x) + b*x + c
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func6, x, y)
            # Predicciones
            y_pred = func6(x, *parametros)

        if Modelo == "Función cociente entre polinomios":
            def func7(x, a, b, c):
                return a*np.abs(x) + b*x + c
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func7, x, y)
            # Predicciones
            y_pred = func7(x, *parametros)

        if Modelo == "Función logaritmica":
            def func8(x, a, b, c):
                return a*np.log(x) + b
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func8, x, y)
            # Predicciones
            y_pred = func8(x, *parametros)

        if Modelo == "Función lineal con producto de coeficientes":
            def func9(x, a, b, c):
                return a*x + b*x + c*x
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func9, x, y)
            # Predicciones
            y_pred = func9(x, *parametros)

        if Modelo == "Función cuadrática inversa":
            def func10(x, a, b, c):
                return 1/a*x**2
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func10, x, y)
            # Predicciones
            y_pred = func10(x, *parametros)

        if Modelo == "Función polinomial inversa":
            def func11(x, a, b, c):
                return a/b*x**2 + c*x
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func11, x, y)
            # Predicciones
            y_pred = func11(x, *parametros)

 
        #Corroboramos cual es el coeficiente de Determinación de nuestro modelo
        coef_Deter = r2_score(y, y_pred) 
        #Corroboramos cual es el coeficiente de Correlación de nuestro modelo
        coef_Correl=np.sqrt(coef_Deter)
        #Mostramos el dataset
        st.write(coef_Correl)
        
        #GRAPH 7: SCATTERPLOT
        figure8 = px.scatter(data_frame=df_superhost, x=Variable_x, y=Variable_y, 
                     title= 'Modelo No Lineal Superhost')

        figure8.add_trace(
            go.Scatter(
                x=df[Variable_x], 
                y=y_pred,
                mode='markers',
                name='Línea de Regresión',
                line=dict(color='red', width=2)
            )
        )
        st.plotly_chart(figure8)
   
   
   
   ##########################################################################################3
        # CONTENIDO DE LA VISTA 4
if View == "Regresión Logística":
    #REGRESIÓN NO LINEAL 
    #Generamos la lista de variables numéricas
    numeric_df = df.select_dtypes(['float','int'])  #Devuelve Columnas
    Lista_num= numeric_df.columns                   #Devuelve lista de Columnas numéricas
    Lista_dicot= ['host_is_superhost','host_has_profile_pic','host_identity_verified','has_availability']                  #Devuelve lista de Columnas dicotómicas
    #Select box
    Variable_y= st.sidebar.selectbox(label= "Variable dependiente (y)", options= Lista_dicot)
    #Multi-select box
    Variables_x= st.sidebar.multiselect(label="Variables independientes del modelo logístico (X)", options= Lista_num,
                                        default=['price'])  

    #Generamos los encabezados para el dashboard
    st.title("Regresión Logística")  
        


    #Generamos el diseño del Layout deseado
    # Fila 1
    Contenedor_A, Contenedor_B = st.columns(2)
    with Contenedor_A: 
        st.write("Correlación Logística General")
        #Redefinimos las variables 
        X= df[Variables_x]
        y= df[Variable_y]
        
        #Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
        
        #Se escalan todos los datos
        escalar = StandardScaler()
        
        #Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
        X_train = escalar.fit_transform(X_train)
        X_test = escalar.transform(X_test)

        #Definimos el algoritmo a utilizar
        from sklearn.linear_model import LogisticRegression
        algoritmo = LogisticRegression()
        
        #Entrenamos el modelo
        algoritmo.fit(X_train, y_train)
        
        #Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
        X_train = escalar.fit_transform(X_train)
        X_test = escalar.transform(X_test)
        
        #Realizamos una predicción
        y_pred = algoritmo.predict(X_test) 
        
        #Verifico la matriz de Confusión
        matriz = confusion_matrix(y_test, y_pred)

        # Etiquetas de los ejes
        clases=  np.unique(df[Variable_y])
        labels = [clases[0], clases[1]]

        # Crear el heatmap
        figure9 = go.Figure(data=go.Heatmap(
            z=matriz,
            x=labels,
            y=labels,
            hoverinfo="z",
            colorscale="Reds",
            showscale=True,
            zmin=0
        ))

        # Añadir anotaciones con TP, TN, FP, FN
        annotations = []
        for i in range(matriz.shape[0]):
            for j in range(matriz.shape[1]):
                valor = matriz[i, j]
                if i == 0 and j == 0:
                    texto = f'TP: {valor}'
                elif i == 0 and j == 1:
                    texto = f'FP: {valor}'
                elif i == 1 and j == 0:
                    texto = f'FN: {valor}'
                elif i == 1 and j == 1:
                    texto = f'TN: {valor}'
                annotations.append(
                    dict(
                        x=labels[j],
                        y=labels[i],
                        text=texto,
                        showarrow=False,
                        font=dict(color="white" if valor > matriz.max()/2 else "black")
                    )
                )

        figure9.update_layout(
            title=f'Matríz de confusión:{Variable_y}',
            xaxis_title="Predicted",
            yaxis_title="Actual",
            annotations=annotations,
            width=500,
            height=500
        )

        # Mostrar en Streamlit
        st.plotly_chart(figure9)

    with Contenedor_B:
        #Calculo la exactitud del modelo
        from sklearn.metrics import accuracy_score
        exactitud = accuracy_score(y_test, y_pred)
        #Mostramos la exactitud del modelo
        st.write("Exactitud del Modelo")
        st.write(exactitud)
        #Calculo la precisión del modelo
        from sklearn.metrics import precision_score
        precision_0 = precision_score(y_test, y_pred, average="binary", pos_label=clases[0])
        st.write("Precisión de la etiqueta:", clases[0])
        st.write(precision_0)
        precision_1 = precision_score(y_test, y_pred, average="binary", pos_label=clases[1])
        st.write("Precisión de la etiqueta:", clases[1])
        st.write(precision_1)
        
 ####################################################################################
 #Referencias
 #Input widgets - Streamlit Docs. (n.d.). https://docs.streamlit.io/develop/api-reference/widgets
 #Star Network. (n.d.). White airbnb icon - Free white site logo icons. https://www.iconsdb.com/white-icons/airbnb-icon.html
 #Get the Data. (s.f.). Inside Airbnb. https://insideairbnb.com/get-the-data/



Overwriting app.py
